In [1]:
import yfinance as yf
from tools import rsi, sma

Source: https://www.youtube.com/watch?v=W8ENIXvcGlQ&t=0s

In [2]:
SYMBOL = "^GSPC"
# SYMBOL = "SPY"

# SYMBOL = "^NDX"
# SYMBOL = "QQQ"

stock = yf.download(SYMBOL)

# some data cleaning
stock = stock[~(stock.High == stock.Low) & ~(stock.Open == stock.Close)]
stock = stock.dropna()

[*********************100%%**********************]  1 of 1 completed


In [23]:
from backtesting import Strategy


class mean_reversion(Strategy):
    """
    strategy for trend_trading
    """

    sma_interval = 200
    rsi_interval = 10
    days = 10
    rsi_buy = 30
    rsi_sell = 40

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )

    def init(self):
        super().init()

        self.rsi = self.I(rsi, self.data.Close.s, self.rsi_interval)

        # regime filter
        self.sma = self.I(sma, self.data.Close.s, self.sma_interval)

    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for existing trade
        for trade in self.trades:
            if trade.is_long:
                duration = len(self.data) - trade.entry_bar

                if duration >= self.days:
                    trade.close()
                elif self.rsi[-1] > self.rsi_sell:
                    trade.close()

        if len(self.trades) == 0:
            if _close < self.sma[-1] and self.rsi[-1] < self.rsi_buy:
                self.buy()

In [24]:
from backtesting import Backtest

bt = Backtest(
    stock["2000-01-01":],
    mean_reversion,
    cash=100_000,
    commission=0.002,
    trade_on_close=True,
)
stats = bt.run()
bt.plot(superimpose=False)
stats

Start                     2000-01-03 00:00:00
End                       2024-05-23 00:00:00
Duration                   8907 days 00:00:00
Exposure Time [%]                    5.477665
Equity Final [$]                135823.739236
Equity Peak [$]                 135823.739236
Return [%]                          35.823739
Buy & Hold Return [%]              261.996121
Return (Ann.) [%]                    1.265841
Volatility (Ann.) [%]                8.803788
Sharpe Ratio                         0.143784
Sortino Ratio                        0.214326
Calmar Ratio                         0.059568
Max. Drawdown [%]                  -21.250261
Avg. Drawdown [%]                   -8.413343
Max. Drawdown Duration     3521 days 00:00:00
Avg. Drawdown Duration      860 days 00:00:00
# Trades                                   58
Win Rate [%]                        79.310345
Best Trade [%]                      12.980407
Worst Trade [%]                    -16.247261
Avg. Trade [%]                    

In [44]:
stats, heatmap = bt.optimize(
    sma_interval=range(50, 300, 25),
    rsi_interval=range(2, 20, 2),
    # days=range(1, 15, 1),
    rsi_buy=range(10, 40, 2),
    rsi_sell=range(10, 80, 3),
    constraint=lambda p: p.rsi_buy < p.rsi_sell,
    maximize="Equity Final [$]",  # "Profit Factor"
    max_tries=500,
    random_state=0,
    return_heatmap=True,
)

/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 384 configurations.
  output = _optimize_grid()
/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "
/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


In [45]:
# heatmap
heatmap.sort_values().dropna().iloc[-10:]

sma_interval  rsi_interval  rsi_buy  rsi_sell
250           4             22       25          200536.107450
150           2             18       19          201257.516244
200           2             18       22          204337.754807
150           2             26       67          205234.829599
50            6             30       73          210216.325249
150           18            38       40          211413.011186
50            12            38       76          213771.207589
150           6             24       25          220095.095661
100           4             28       76          233298.300480
50            10            32       46          265326.171292
Name: Equity Final [$], dtype: float64

In [46]:
# Recommendation
stats["_strategy"]

<Strategy mean_reversion(sma_interval=50,rsi_interval=10,rsi_buy=32,rsi_sell=46)>

In [47]:
stats = bt.run(**stats._strategy._params)
bt.plot(superimpose=False, open_browser=True)
stats

Start                     2000-01-03 00:00:00
End                       2024-05-23 00:00:00
Duration                   8907 days 00:00:00
Exposure Time [%]                    12.84643
Equity Final [$]                265326.171292
Equity Peak [$]                 265326.171292
Return [%]                         165.326171
Buy & Hold Return [%]              261.996121
Return (Ann.) [%]                    4.090224
Volatility (Ann.) [%]               11.147866
Sharpe Ratio                         0.366906
Sortino Ratio                        0.586829
Calmar Ratio                         0.179461
Max. Drawdown [%]                  -22.791739
Avg. Drawdown [%]                   -3.479179
Max. Drawdown Duration     2768 days 00:00:00
Avg. Drawdown Duration      113 days 00:00:00
# Trades                                  108
Win Rate [%]                        69.444444
Best Trade [%]                      17.315658
Worst Trade [%]                    -16.247261
Avg. Trade [%]                    